In [7]:
! wget https://github.com/erkansirin78/datasets/raw/master/dirty_store_transactions.csv

--2024-10-19 18:43:55--  https://github.com/erkansirin78/datasets/raw/master/dirty_store_transactions.csv
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/erkansirin78/datasets/master/dirty_store_transactions.csv [following]
--2024-10-19 18:43:56--  https://raw.githubusercontent.com/erkansirin78/datasets/master/dirty_store_transactions.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2609524 (2.5M) [text/plain]
Saving to: ‘dirty_store_transactions.csv’

dirty_store_transac 100%[===================>]   2.49M  4.91MB/s    in 0.5s    

2024-10-19 18:43:57 (4.91 MB/s) - ‘dirty_store_transactions.c

In [ ]:
### 1. 
Clean data till you get following schema.
#### 1.1. Schema
```text
root
 |-- STORE_ID: string (nullable = true)
 |-- STORE_LOCATION: string (nullable = true)
 |-- PRODUCT_CATEGORY: string (nullable = true)
 |-- PRODUCT_ID: integer (nullable = true)
 |-- MRP: float (nullable = true)
 |-- CP: float (nullable = true)
 |-- DISCOUNT: float (nullable = true)
 |-- SP: float (nullable = true)
 |-- Date_Casted: date (nullable = true)

In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F

In [3]:
spark = SparkSession.builder \
.appName("week7") \
.master("local[2]") \
.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/19 18:42:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [47]:
df = spark.read.format("csv").option("header", True).load("file:///opt/examples/dirty_store_transactions.csv")

In [48]:
df.count()

37853

In [49]:
df.show()

+--------+--------------+----------------+----------+---+------+--------+------+----------+
|STORE_ID|STORE_LOCATION|PRODUCT_CATEGORY|PRODUCT_ID|MRP|    CP|DISCOUNT|    SP|      Date|
+--------+--------------+----------------+----------+---+------+--------+------+----------+
|  YR7220|     New York(|     Electronics|  12254943|$31|$20.77|   $1.86|$29.14|2019-11-26|
|  YR7220|     New York+|       Furniture| 72619323C|$15| $9.75|    $1.5| $13.5|2019-11-26|
|  YR7220|     New York |     Electronics| 34161682B|$88|$62.48|    $4.4| $83.6|2019-11-26|
|  YR7220|     New York!|         Kitchen|  79411621|$91|$58.24|   $3.64|$87.36|2019-11-26|
|  YR7220|      New York|         Fashion| 39520263T|$85|   $51|   $2.55|$82.45|2019-11-26|
|  YR7220|      New York|         Kitchen|  93809204|$37|$24.05|   $0.74|$36.26|2019-11-26|
|  YR7220|      New York|       Cosmetics| 86610412D|$80| $48.8|    $6.4| $73.6|2019-11-26|
|  YR7220|     New York$|         Kitchen| 52503356^|$71| $42.6|   $5.68|$65.32|

In [50]:
df.printSchema()

root
 |-- STORE_ID: string (nullable = true)
 |-- STORE_LOCATION: string (nullable = true)
 |-- PRODUCT_CATEGORY: string (nullable = true)
 |-- PRODUCT_ID: string (nullable = true)
 |-- MRP: string (nullable = true)
 |-- CP: string (nullable = true)
 |-- DISCOUNT: string (nullable = true)
 |-- SP: string (nullable = true)
 |-- Date: string (nullable = true)



In [51]:
df = df.select(*[F.trim(F.col(c)).alias(c) if isinstance(df.schema[c].dataType, StringType) else col(c) for c in df.columns])

In [52]:
df.show()

+--------+--------------+----------------+----------+---+------+--------+------+----------+
|STORE_ID|STORE_LOCATION|PRODUCT_CATEGORY|PRODUCT_ID|MRP|    CP|DISCOUNT|    SP|      Date|
+--------+--------------+----------------+----------+---+------+--------+------+----------+
|  YR7220|     New York(|     Electronics|  12254943|$31|$20.77|   $1.86|$29.14|2019-11-26|
|  YR7220|     New York+|       Furniture| 72619323C|$15| $9.75|    $1.5| $13.5|2019-11-26|
|  YR7220|      New York|     Electronics| 34161682B|$88|$62.48|    $4.4| $83.6|2019-11-26|
|  YR7220|     New York!|         Kitchen|  79411621|$91|$58.24|   $3.64|$87.36|2019-11-26|
|  YR7220|      New York|         Fashion| 39520263T|$85|   $51|   $2.55|$82.45|2019-11-26|
|  YR7220|      New York|         Kitchen|  93809204|$37|$24.05|   $0.74|$36.26|2019-11-26|
|  YR7220|      New York|       Cosmetics| 86610412D|$80| $48.8|    $6.4| $73.6|2019-11-26|
|  YR7220|     New York$|         Kitchen| 52503356^|$71| $42.6|   $5.68|$65.32|

In [53]:
df.select("product_category").distinct().show()

+----------------+
|product_category|
+----------------+
|       Education|
|         Kitchen|
|         Fashion|
|       Groceries|
|     Electronics|
|       Furniture|
|       Cosmetics|
+----------------+



In [54]:
df.groupBy("store_location").count().show()

+--------------+-----+
|store_location|count|
+--------------+-----+
|        Miami'|  222|
|    New York""|  315|
|  Washington""|  348|
|        Miami)|  282|
|      Houston%|  252|
|        Miami&|  294|
|      Houston$|  245|
|     New York$|  245|
|     New York'|  371|
|     New York+|  266|
|     New York(|  343|
|     New York!|  364|
|        Miami#|  288|
|      Houston*|  280|
|     New York&|  252|
|        Miami$|  294|
|       Denver$|  228|
|      Houston'|  308|
|       Denver+|  228|
|      Houston(|  357|
+--------------+-----+
only showing top 20 rows



In [55]:
df = df.withColumn("STORE_LOCATION", F.regexp_replace(F.col("store_location"), r"[^a-zA-Z0-9\s]", ""))

In [56]:
df.groupBy("store_location").count().show()

+--------------+-----+
|store_location|count|
+--------------+-----+
|    Washington| 8604|
|       Houston| 7385|
|         Miami| 7254|
|      New York| 8064|
|        Denver| 6546|
+--------------+-----+



In [57]:
df = df.withColumn("PRODUCT_ID",F.regexp_replace(F.col("product_id"), r"[^0-9]", "")) \
       .withColumn("PRODUCT_ID", F.col("product_id").cast("int"))

In [58]:
df.printSchema()

root
 |-- STORE_ID: string (nullable = true)
 |-- STORE_LOCATION: string (nullable = true)
 |-- PRODUCT_CATEGORY: string (nullable = true)
 |-- PRODUCT_ID: integer (nullable = true)
 |-- MRP: string (nullable = true)
 |-- CP: string (nullable = true)
 |-- DISCOUNT: string (nullable = true)
 |-- SP: string (nullable = true)
 |-- Date: string (nullable = true)



In [59]:
float_cols = ["MRP", "CP", "DISCOUNT", "SP"]

In [60]:
for column in float_cols:
    df = df.withColumn(column, F.regexp_replace(F.col(column), r"\$", "")) \
           .withColumn(column, F.col(column).cast("float"))

In [61]:
df.show()

+--------+--------------+----------------+----------+----+-----+--------+-----+----------+
|STORE_ID|STORE_LOCATION|PRODUCT_CATEGORY|PRODUCT_ID| MRP|   CP|DISCOUNT|   SP|      Date|
+--------+--------------+----------------+----------+----+-----+--------+-----+----------+
|  YR7220|      New York|     Electronics|  12254943|31.0|20.77|    1.86|29.14|2019-11-26|
|  YR7220|      New York|       Furniture|  72619323|15.0| 9.75|     1.5| 13.5|2019-11-26|
|  YR7220|      New York|     Electronics|  34161682|88.0|62.48|     4.4| 83.6|2019-11-26|
|  YR7220|      New York|         Kitchen|  79411621|91.0|58.24|    3.64|87.36|2019-11-26|
|  YR7220|      New York|         Fashion|  39520263|85.0| 51.0|    2.55|82.45|2019-11-26|
|  YR7220|      New York|         Kitchen|  93809204|37.0|24.05|    0.74|36.26|2019-11-26|
|  YR7220|      New York|       Cosmetics|  86610412|80.0| 48.8|     6.4| 73.6|2019-11-26|
|  YR7220|      New York|         Kitchen|  52503356|71.0| 42.6|    5.68|65.32|2019-11-26|

In [63]:
df = df.withColumn("Date_Casted", F.col("Date").cast("date")).drop("Date")

In [64]:
df.show()

+--------+--------------+----------------+----------+----+-----+--------+-----+-----------+
|STORE_ID|STORE_LOCATION|PRODUCT_CATEGORY|PRODUCT_ID| MRP|   CP|DISCOUNT|   SP|Date_Casted|
+--------+--------------+----------------+----------+----+-----+--------+-----+-----------+
|  YR7220|      New York|     Electronics|  12254943|31.0|20.77|    1.86|29.14| 2019-11-26|
|  YR7220|      New York|       Furniture|  72619323|15.0| 9.75|     1.5| 13.5| 2019-11-26|
|  YR7220|      New York|     Electronics|  34161682|88.0|62.48|     4.4| 83.6| 2019-11-26|
|  YR7220|      New York|         Kitchen|  79411621|91.0|58.24|    3.64|87.36| 2019-11-26|
|  YR7220|      New York|         Fashion|  39520263|85.0| 51.0|    2.55|82.45| 2019-11-26|
|  YR7220|      New York|         Kitchen|  93809204|37.0|24.05|    0.74|36.26| 2019-11-26|
|  YR7220|      New York|       Cosmetics|  86610412|80.0| 48.8|     6.4| 73.6| 2019-11-26|
|  YR7220|      New York|         Kitchen|  52503356|71.0| 42.6|    5.68|65.32| 

In [65]:
df.printSchema()

root
 |-- STORE_ID: string (nullable = true)
 |-- STORE_LOCATION: string (nullable = true)
 |-- PRODUCT_CATEGORY: string (nullable = true)
 |-- PRODUCT_ID: integer (nullable = true)
 |-- MRP: float (nullable = true)
 |-- CP: float (nullable = true)
 |-- DISCOUNT: float (nullable = true)
 |-- SP: float (nullable = true)
 |-- Date_Casted: date (nullable = true)

